In [0]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DoubleType, BooleanType, DateType

In [0]:
# Use SAS instead

sas_token = "sp=racwdlmeop&st=2023-10-17T16:59:59Z&se=2023-10-18T00:59:59Z&spr=https&sv=2022-11-02&sr=c&sig=7GYCZyR0Krk6WlVM6OniZXMmd%2FZgi4Q0BaaEblbYhtQ%3D"

conf_key = f"fs.azure.sas.tokyo-olympic-data.rwadls.blob.core.windows.net"


try:
  dbutils.fs.mount(
    source = f"wasbs://tokyo-olympic-data@rwadls.blob.core.windows.net/",
    mount_point = f"/mnt/tokyoolympic",
    extra_configs = {conf_key: sas_token})
except:
  pass


In [0]:
%fs
ls "/mnt/tokyoolympic"

In [0]:
spark

In [0]:
athletes = spark.read.format("csv") \
  .option("inferSchema", "true") \
  .option("header", "true") \
  .load("/mnt/tokyoolympic/raw-data/athletes.csv")

coaches = spark.read.format("csv") \
  .option("inferSchema", "true") \
  .option("header", "true") \
  .load("/mnt/tokyoolympic/raw-data/coaches.csv")

entries_gender = spark.read.format("csv") \
  .option("inferSchema", "true") \
  .option("header", "true") \
  .load("/mnt/tokyoolympic/raw-data/entries_gender.csv")

medals = spark.read.format("csv") \
  .option("inferSchema", "true") \
  .option("header", "true") \
  .load("/mnt/tokyoolympic/raw-data/medals.csv")

teams = spark.read.format("csv") \
  .option("inferSchema", "true") \
  .option("header", "true") \
  .load("/mnt/tokyoolympic/raw-data/teams.csv")


In [0]:
# show data
athletes.show(10)

# understand the schema
athletes.printSchema()

# show data
coaches.show(10)

# understand the schema
coaches.printSchema()

entries_gender.show(10)
entries_gender.printSchema()

medals.show(10)
medals.printSchema()

teams.show(10)
teams.printSchema()

In [0]:
entries_gender = entries_gender.withColumn("Female", col("Female").cast(IntegerType()))\
    .withColumn("Male", col("Male").cast(IntegerType())) \
    .withColumn("Total", col("Total").cast(IntegerType()))

In [0]:
top_gold_medal_countries = medals.orderBy("Gold", ascending=False).show()

In [0]:
average_entries_by_gender = entries_gender.withColumn(
    'Avg_Female', entries_gender['Female'] / entries_gender['Total']
).withColumn(
    'Avg_Male', entries_gender['Male'] / entries_gender['Total']
)
average_entries_by_gender.show()

In [0]:
athletes.repartition(1).write.mode("overwrite").option("header",'true').csv("/mnt/tokyoolympic/transformed-data/athletes")
coaches.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/tokyoolympic/transformed-data/coaches")
entries_gender.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/tokyoolympic/transformed-data/entries_gender")
medals.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/tokyoolympic/transformed-data/medals")
teams.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/tokyoolympic/transformed-data/teams")

In [0]:
%fs ls "/mnt/tokyoolympic/transformed-data"